<a href="https://colab.research.google.com/github/BankNatchapol/Comparison-Of-Quantum-Gradient/blob/main/concept_implementation/newton_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit
!pip install pennylane

In [2]:
#General imports
import numpy as np

#Operator Imports
from qiskit.aqua.operators import Z, X, I, StateFn, CircuitStateFn, SummedOp
from qiskit.aqua.operators.gradients import Gradient, Hessian

#Circuit imports
from qiskit.circuit import QuantumCircuit, QuantumRegister, Parameter, ParameterVector, ParameterExpression

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


# **Verifying Hessian metric**

In [3]:
# Instantiate the Hamiltonian observable
H = 0.5*X - 1.0*Z

# Instantiate the quantum state with two parameters
a = Parameter('a')
b = Parameter('b')
c = Parameter('c')

q = QuantumRegister(1)
qc = QuantumCircuit(q)
qc.h(q)
qc.rx(a, q[0])
qc.ry(b, q[0])
qc.rz(c, q[0])


# Combine the Hamiltonian observable and the state
op = ~StateFn(H) @ CircuitStateFn(primitive=qc, coeff=1.)

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/operators/operator_base.py:46: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua.operators', 'qiskit.opflow', 'qiskit-terra')


In [4]:
# Define parameters
params = [a, b, c]
value_dict = {a: np.pi / 4, b: np.pi/4, c: np.pi/4}
state_hess = Hessian(hess_method='param_shift').convert(operator=op, params=params)
hessian_result_circuit = state_hess.assign_parameters(value_dict)
print(hessian_result_circuit)

ListOp([
  ListOp([
    SummedOp([
      0.5 * SummedOp([
        SummedOp([
          0.25 * ComposedOp([
            OperatorMeasurement(Z),
            CircuitStateFn(
                  ┌───┐┌──────────┐┌─────────┐┌─────────┐┌───┐
            q0_0: ┤ H ├┤ Rx(5π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├┤ H ├
                  └───┘└──────────┘└─────────┘└─────────┘└───┘
            )
          ]),
          -0.25 * ComposedOp([
            OperatorMeasurement(Z),
            CircuitStateFn(
                  ┌───┐┌─────────┐┌─────────┐┌─────────┐┌───┐
            q0_0: ┤ H ├┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├┤ H ├
                  └───┘└─────────┘└─────────┘└─────────┘└───┘
            )
          ])
        ]),
        SummedOp([
          -0.25 * ComposedOp([
            OperatorMeasurement(Z),
            CircuitStateFn(
                  ┌───┐┌─────────┐┌─────────┐┌─────────┐┌───┐
            q0_0: ┤ H ├┤ Rx(π/4) ├┤ Ry(π/4) ├┤ Rz(π/4) ├┤ H ├
                  └───┘└─────────┘└─────────┘└────────

In [5]:
hessian_result = np.array(hessian_result_circuit.eval())
print('Hessian computed with finite difference\n', str(hessian_result).replace("       ", "").replace("j ", "j, "))

Hessian computed with finite difference
 [[-1.38777878e-17+3.2875e-17j,  2.77555756e-17+7.2750e-17j
   6.93889390e-17-2.9375e-17j]
 [ 1.38777878e-17+7.2750e-17j, -9.57106781e-01-9.0500e-17j
   2.50000000e-01+9.9000e-17j]
 [ 6.93889390e-17-2.9375e-17j,  2.50000000e-01+9.9000e-17j
  -2.50000000e-01-1.4000e-17j]]


In [6]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer

import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [7]:
num_wires = 1
dev = qml.device("default.qubit", wires=num_wires)

In [8]:
s = np.pi/2

def parameter_shift_term(qnode, params, i, j):
    
    shifted = params.copy()
    
    shifted[i, j] += np.pi/2
    forward = qnode(shifted)  # forward evaluation

    shifted[i, j] -= np.pi
    backward = qnode(shifted) # backward evaluation

    return 0.5 * (forward - backward)

def parameter_shift(qnode, params):
    gradients = np.zeros_like((params))
    for i in range(len(gradients)):
        for j in range(len(gradients[0])):
            gradients[i, j] += parameter_shift_term(qnode, params, i, j)

    return gradients

def hessian_parameter_shift_term(qnode, params, i, j):
    
    shifted = params.copy()
    
    shifted[i, j] += 2*s
    forward = qnode(shifted)  # forward evaluation

    shifted[i, j] -= 2*s
    center = qnode(shifted) # center evaluation
    
    shifted[i, j] -= 2*s
    backward = qnode(shifted) # backward evaluation

    return (1/4) * (forward - 2*center + backward)

def hessian_matrix(qnode, params):
    hessian_m = np.zeros_like((params))
    for i in range(len(hessian_m)):
        for j in range(len(hessian_m[0])):
            hessian_m[i, j] += hessian_parameter_shift_term(qnode, params, i, j)
    hessian = np.diag(hessian_m.flatten())
    hessian[hessian == 0] = 1e-17
    return hessian

In [9]:
# guesting ansatz state
def ansatz(var):
    for wire in range(num_wires):
      qml.Hadamard(wires=wire)
      qml.RX(var[0+wire], wires=wire)
      qml.RY(var[1+wire], wires=wire)
      qml.RZ(var[2+wire], wires=wire)

In [10]:
coeffs = [0.5, -1.0]
H = [qml.PauliX(wires=0), qml.PauliZ(wires=0)]
hamiltonian = qml.Hamiltonian(coeffs, H)

@qml.qnode(dev)
def cost_function(var):
  for v in var:
    ansatz(v)
  return qml.expval(hamiltonian)

In [11]:
params = np.array([[np.pi / 4, np.pi/4,  np.pi/4]])
hessian_result_pennylane = hessian_matrix(cost_function, params)
print("Hessian metric using Qiskit    :\n", hessian_result)
print()
print("Hessian metric using Pennylane :\n", hessian_result_pennylane)

Hessian metric using Qiskit    :
 [[-1.38777878e-17+3.2875e-17j  2.77555756e-17+7.2750e-17j
   6.93889390e-17-2.9375e-17j]
 [ 1.38777878e-17+7.2750e-17j -9.57106781e-01-9.0500e-17j
   2.50000000e-01+9.9000e-17j]
 [ 6.93889390e-17-2.9375e-17j  2.50000000e-01+9.9000e-17j
  -2.50000000e-01-1.4000e-17j]]

Hessian metric using Pennylane :
 [[ 2.77555756e-17  1.00000000e-17  1.00000000e-17]
 [ 1.00000000e-17 -9.57106781e-01  1.00000000e-17]
 [ 1.00000000e-17  1.00000000e-17 -2.50000000e-01]]


# **Quantum gate estimation using Newton gradient with parameter shift rule**

## Newton gradient
using parameter shift rule to find first-order gradient of cost function $\nabla J(\theta)$<br>
matrix multiplication with inverse hessian metric to get second-order gradient
<br><br>
$$Second\  order\ gradient = Hessian^{-1} \nabla J(\theta)$$
<br>
using np.linalg.solve to find solve the equation<br><br>
$$ Hessian\ x = \nabla J(\theta)$$

In [12]:
def newton_gradient(qnode, params):
  var = params.copy()
  hessian = hessian_matrix(qnode, var)
  grad = parameter_shift(qnode, var).flatten()
  newton_grad = np.linalg.solve(hessian, grad)
  return newton_grad

## Quantum gate estimation
Using $U3(1.44, 0.8, 2.1)$ as target gate

In [13]:
# problem gate 
def problem():
    qml.U3(1.44, 0.8, 2.1, wires=0)

Ansatz for estimating target gate

In [14]:
# guesting ansatz state
def ansatz(var):
    for wire in range(num_wires):
      qml.Hadamard(wires=wire)
      qml.RX(var[0+wire], wires=wire)
      qml.RY(var[1+wire], wires=wire)
      qml.RZ(var[2+wire], wires=wire)

Objective function <br> 
1. initial state is  $|0⟩$
2. apply target gate $U(\theta)|0⟩$ <br> 
the state will be $|\psi⟩ = a|0⟩+b|1⟩$ 
3. apply ansatz $A(\alpha)$  <br> 
the state will be $A(\alpha)|\psi⟩ = |\psi'⟩$
4. if $A(\alpha) = U(\theta)$ then  $|\psi'⟩= 1|0⟩ + 0|1⟩ = |0⟩$
5. so, we will minimize $b|1⟩$ to target $0|1⟩$ to make $A(\alpha) = U(\theta)$


In [15]:
# objective function
@qml.qnode(dev)
def cost_function(var):
    for v in var: 
      ansatz(v)

    problem() # problem gate 

    return qml.expval(qml.Projector([1],wires=0)) # get amplitude of of |1>

Probability distribution of target gate 

In [16]:
# target result of problem gate
@qml.qnode(dev)
def target():
    problem()
    return qml.probs(wires=[0])  # get target probability

Probability distribution of estimated gate 

In [17]:
# prediction circuit
@qml.qnode(dev)
def prediction(var):
    for v in reversed(var):
      qml.adjoint(ansatz)(v)
    return qml.probs(wires=[0])  # get prediction probability

In [18]:
print("Target state: ", target())

Target state:  [0.56521185 0.43478815]


In [19]:
np.random.seed(1)
num_layers = 2
var_init = 0.05*np.random.randn(num_layers, 3*num_wires)

In [20]:
opt = GradientDescentOptimizer(0.01)

var = var_init.copy()
loss_plot = []

for it in range(501):# while True:
    var, _cost = opt.step_and_cost(lambda v: cost_function(v), var, 
                                   grad_fn=lambda var: newton_gradient(cost_function, var)) 
    loss_plot.append(_cost)

    if it%100==0:
      print("Iter: {:5d} | Cost: {:0.11f} ".format(it, _cost))

Iter:     0 | Cost: 0.38420647954 
Iter:   100 | Cost: 0.00147238594 
Iter:   200 | Cost: 0.00001017423 
Iter:   300 | Cost: 0.00000009168 
Iter:   400 | Cost: 0.00000000095 
Iter:   500 | Cost: 0.00000000001 


In [21]:
#@title 
fig = px.line(pd.DataFrame({"Iteration":range(len(loss_plot)), "Loss":loss_plot}), 
                          x="Iteration", y="Loss", title="Newton optimizer loss", width=1000)
fig.show()

In [22]:
print("Target probs    : ", target())
print("Prediction probs: ", prediction(var))

Target probs    :  [0.56521185 0.43478815]
Prediction probs:  [0.56521457 0.43478543]
